### Exploration API Mistral

In [2]:
from config import MISTRALAI_API_KEY
from mistralai import Mistral
import pandas as pd

model = "mistral-small-latest"
client = Mistral(api_key=MISTRALAI_API_KEY)

In [ ]:
sample_paper_text = pd.read_csv('../csvs/Lookingbill_Wagner_2025.csv')['body'].tolist()[2]

'INTRODUCTION\nSocial media platforms are often celebrated for their capacity to connect. However, when people experience severe distress, trauma, or stigmatized experiences (e.g., pregnancy loss, HIV, terminal illness diagnosis), they may find it difficult to use these platforms to talk about such experiences and feelings associated with them, or seek support; yet the need to disclose these experiences often remains [91]. The potential to receive social support often motivates people to make sensitive self-disclosures [45]. Support can take multiple forms [27], and receiving helpful social support after sensitive disclosures are made is linked to improved wellbeing [74]. Sexual abuse is one important example of a highly stigmatized human experience that is difficult to disclose and seek support around, but for which receiving support is vital for many [105].\nAbuse survivors often fear receiving unsupportive reactions and rejection from confidants [53,59]. However, disclosing these ex

In [ ]:
specific_data_elements = {
    'Methodology' : 'Whether the study followed a qualitative, quantitative or mixed methods research design',
    'Sample' : 'Size and description of the sample used in the study',
    'ICT' : 'The Information and Communication Technologies (ICTs) used by the participants of the study. ICTs include search engines, social network sites, and smartphones',
    'Framing of disclosure' : 'A category designating the way that the article surface ICT-based disclosure'
}

system_prompt = f"""You are an expert in data extraction for literature reviews in the social sciences.
You extract data from the paper provided by the user, based on the data elements specified below:

{specific_data_elements}. 
"""

In [ ]:
user_prompt = 

In [ ]:
chat_response = client.chat.complete(
    model = model,
    messages = [
        {
            "role": "system",
            "content": "You are an expert in data extraction for literature reviews",
        },
        {
            "role": "user",
            "content": f"What is the research method used in this text\n\{sample_paper}",
        },
    ]
)

print(chat_response.choices[0].message.content)

The research method used in the provided text is primarily **content analysis**, which is a systematic and objective means of describing and quantifying phenomena. Here's a breakdown of the methods used:

1. **Content Analysis**:
   - **Manual and Automated Coding**: The study employs both manual and automated techniques to code tweet content. This involves categorizing and quantifying the content of tweets to identify patterns, themes, and trends.
   - **Text Mining**: The use of text-mining tools like PamTAT and Harvard General Inquirer to analyze large volumes of text data. This includes frequency analysis of unigrams, bigrams, and trigrams, as well as semantic and sentiment analysis.

2. **Data Collection**:
   - **Twitter Data**: The dataset was collected from the complete Twitter timelines of 10 academic libraries, including all tweets since they joined the platform. The data was retrieved using an archiving service (twimemachine.com) in December 2014.
   - **Metadata**: In addit